In [1]:
import pandas as pd

train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [2]:
train.shape

(26457, 20)

In [3]:
test.shape

(10000, 19)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26457 entries, 0 to 26456
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          26457 non-null  int64  
 1   gender         26457 non-null  object 
 2   car            26457 non-null  object 
 3   reality        26457 non-null  object 
 4   child_num      26457 non-null  int64  
 5   income_total   26457 non-null  float64
 6   income_type    26457 non-null  object 
 7   edu_type       26457 non-null  object 
 8   family_type    26457 non-null  object 
 9   house_type     26457 non-null  object 
 10  DAYS_BIRTH     26457 non-null  int64  
 11  DAYS_EMPLOYED  26457 non-null  int64  
 12  FLAG_MOBIL     26457 non-null  int64  
 13  work_phone     26457 non-null  int64  
 14  phone          26457 non-null  int64  
 15  email          26457 non-null  int64  
 16  occyp_type     18286 non-null  object 
 17  family_size    26457 non-null  float64
 18  begin_

In [5]:
train.isnull().sum()

index               0
gender              0
car                 0
reality             0
child_num           0
income_total        0
income_type         0
edu_type            0
family_type         0
house_type          0
DAYS_BIRTH          0
DAYS_EMPLOYED       0
FLAG_MOBIL          0
work_phone          0
phone               0
email               0
occyp_type       8171
family_size         0
begin_month         0
credit              0
dtype: int64

In [6]:
train=train.drop('occyp_type', axis=1)
test=test.drop('occyp_type', axis=1)

In [7]:
unique_len = train.apply(lambda x : len(x.unique()))

unique_len

index            26457
gender               2
car                  2
reality              2
child_num            9
income_total       249
income_type          5
edu_type             5
family_type          5
house_type           6
DAYS_BIRTH        6621
DAYS_EMPLOYED     3470
FLAG_MOBIL           1
work_phone           2
phone                2
email                2
family_size         10
begin_month         61
credit               3
dtype: int64

In [8]:
train=train.drop('index', axis=1)
test=test.drop('index', axis=1)

train=train.drop('FLAG_MOBIL', axis=1)
test=test.drop('FLAG_MOBIL', axis=1)

In [9]:
train['gender'] = train['gender'].replace(['F','M'],[0,1])
test['gender'] = test['gender'].replace(['F','M'],[0,1])
print(train['gender'].value_counts())

train['car'] = train['car'].replace(['N','Y'],[0,1])
test['car'] = test['car'].replace(['N','Y'],[0,1])
print(train['car'].value_counts())

train['reality'] = train['reality'].replace(['N','Y'],[0,1])
test['reality'] = test['reality'].replace(['N','Y'],[0,1])
print(train['reality'].value_counts())

print(train['phone'].value_counts())

print(train['email'].value_counts())    

print(train['work_phone'].value_counts())

0    17697
1     8760
Name: gender, dtype: int64
0    16410
1    10047
Name: car, dtype: int64
1    17830
0     8627
Name: reality, dtype: int64
0    18672
1     7785
Name: phone, dtype: int64
0    24042
1     2415
Name: email, dtype: int64
0    20511
1     5946
Name: work_phone, dtype: int64


In [10]:
train=train.drop('child_num', axis=1)
test=test.drop('child_num', axis=1)
print(train['family_size'].value_counts())

2.0     14106
1.0      5109
3.0      4632
4.0      2260
5.0       291
6.0        44
7.0         9
15.0        3
9.0         2
20.0        1
Name: family_size, dtype: int64


In [11]:
quartiles = train['family_size'].quantile([0.25, 0.5, 0.75])
print(quartiles)

0.25    2.0
0.50    2.0
0.75    3.0
Name: family_size, dtype: float64


In [12]:
train.loc[train['family_size'] >= 4,'family_size']=4
test.loc[test['family_size']>=4, 'family_size']=4
print(train['family_size'].value_counts())

2.0    14106
1.0     5109
3.0     4632
4.0     2610
Name: family_size, dtype: int64


In [13]:
def days_to_year(x):
    if x<0:
        return (x*(-1))/365
    return 0

def minus_to_plus(x):
    if x<0:
        return x*(-1)
    return 0

In [14]:
train['DAYS_BIRTH'] = train['DAYS_BIRTH'].apply(days_to_year)
train['DAYS_EMPLOYED'] = train['DAYS_EMPLOYED'].apply(days_to_year)
train['begin_month'] = train['begin_month'].apply(minus_to_plus)

test['DAYS_BIRTH'] = test['DAYS_BIRTH'].apply(days_to_year)
test['DAYS_EMPLOYED'] = test['DAYS_EMPLOYED'].apply(days_to_year)
test['begin_month'] = test['begin_month'].apply(minus_to_plus)

In [15]:
#생략 가능

train['family_size'] = train['family_size'].astype('int64')
test['family_size'] = test['family_size'].astype('int64')

train['begin_month'] = train['begin_month'].astype('int64')
test['begin_month'] = test['begin_month'].astype('int64')

In [16]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)

income_type_encoded = encoder.fit_transform(train[['income_type']])
income_type_encoded_df = pd.DataFrame(income_type_encoded, columns=encoder.get_feature_names_out(['income_type']))
train = pd.concat([train, income_type_encoded_df], axis=1).drop('income_type', axis=1)

income_type_encoded_test = encoder.transform(test[['income_type']])
income_type_encoded_test_df = pd.DataFrame(income_type_encoded_test, columns=encoder.get_feature_names_out(['income_type']))
test = pd.concat([test, income_type_encoded_test_df], axis=1).drop('income_type', axis=1)

encoder = OneHotEncoder(sparse_output=False)
edu_type_encoded = encoder.fit_transform(train[['edu_type']])
edu_type_encoded_df = pd.DataFrame(edu_type_encoded, columns=encoder.get_feature_names_out(['edu_type']))
train = pd.concat([train, edu_type_encoded_df], axis=1).drop('edu_type', axis=1)

edu_type_encoded_test = encoder.transform(test[['edu_type']])
edu_type_encoded_test_df = pd.DataFrame(edu_type_encoded_test, columns=encoder.get_feature_names_out(['edu_type']))
test = pd.concat([test, edu_type_encoded_test_df], axis=1).drop('edu_type', axis=1)

encoder = OneHotEncoder(sparse_output=False)
family_type_encoded = encoder.fit_transform(train[['family_type']])
family_type_encoded_df = pd.DataFrame(family_type_encoded, columns=encoder.get_feature_names_out(['family_type']))
train = pd.concat([train, family_type_encoded_df], axis=1).drop('family_type', axis=1)

family_type_encoded_test = encoder.transform(test[['family_type']])
family_type_encoded_test_df = pd.DataFrame(family_type_encoded_test, columns=encoder.get_feature_names_out(['family_type']))
test = pd.concat([test, family_type_encoded_test_df], axis=1).drop('family_type', axis=1)

encoder = OneHotEncoder(sparse_output=False)
house_type_encoded = encoder.fit_transform(train[['house_type']])
house_type_encoded_df = pd.DataFrame(house_type_encoded, columns=encoder.get_feature_names_out(['house_type']))
train = pd.concat([train, house_type_encoded_df], axis=1).drop('house_type', axis=1)

house_type_encoded_test = encoder.transform(test[['house_type']])
house_type_encoded_test_df = pd.DataFrame(house_type_encoded_test, columns=encoder.get_feature_names_out(['house_type']))
test = pd.concat([test, house_type_encoded_test_df], axis=1).drop('house_type', axis=1)

In [17]:
train.head()

,gender,car,reality,income_total,DAYS_BIRTH,DAYS_EMPLOYED,work_phone,phone,email,family_size,...,family_type_Married,family_type_Separated,family_type_Single / not married,family_type_Widow,house_type_Co-op apartment,house_type_House / apartment,house_type_Municipal apartment,house_type_Office apartment,house_type_Rented apartment,house_type_With parents
0,0,0,0,202500.0,38.079452,12.901370,0,0,0,2,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0,0,1,247500.0,31.178082,4.219178,0,0,1,3,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,1,1,450000.0,52.293151,12.147945,0,1,0,2,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0,0,1,202500.0,41.336986,5.731507,0,1,0,2,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0,1,1,157500.0,41.197260,5.767123,0,0,0,2,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

X = train.drop(columns=['credit'])  
y = train['credit']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# 예측값을 0, 1, 2 범주로 변환
def convert_to_category(predictions, threshold):
    categories = []
    for pred in predictions:
        if pred <= threshold[0]:
            categories.append(0)
        elif pred <= threshold[1]:
            categories.append(1)
        else:
            categories.append(2)
    return categories

# 임계값
threshold = [0.5, 1.5]  # 예시 임계값

y_pred_categories = convert_to_category(y_pred, threshold)

accuracy = accuracy_score(y_test, y_pred_categories)
print("정확성: ", accuracy)

정확성:  0.5081254724111867


In [19]:
import numpy as np

def generate_random_threshold():
    return np.sort(np.random.uniform(low=0, high=2, size=2))

np.random.seed(42)

best_accuracy = 0.0
best_threshold = None

for _ in range(100):  
    threshold = generate_random_threshold()

    y_pred_categories = convert_to_category(y_pred, threshold)

    accuracy = accuracy_score(y_test, y_pred_categories)

    print("임계값:", threshold, "정확성:", accuracy)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold

print("가장 높은 정확성:", best_accuracy)
print("가장 높은 정확성을 가진 임계값 조합:", best_threshold)

임계값: [0.74908024 1.90142861] 정확성: 0.23847316704459562
임계값: [1.19731697 1.46398788] 정확성: 0.5636810279667422
임계값: [0.31198904 0.31203728] 정확성: 0.6386999244142101
임계값: [0.11616722 1.73235229] 정확성: 0.25321239606953894
임계값: [1.20223002 1.41614516] 정확성: 0.6294406651549509
임계값: [0.04116899 1.9398197 ] 정확성: 0.23847316704459562
임계값: [0.42467822 1.66488528] 정확성: 0.3063114134542706
임계값: [0.36364993 0.36680902] 정확성: 0.6386999244142101
임계값: [0.60848449 1.04951286] 정확성: 0.6386999244142101
임계값: [0.58245828 0.86389004] 정확성: 0.6386999244142101
임계값: [0.27898772 1.22370579] 정확성: 0.6386999244142101
임계값: [0.5842893  0.73272369] 정확성: 0.6386999244142101
임계값: [0.91213997 1.57035192] 정확성: 0.4145880574452003
임계값: [0.39934756 1.02846888] 정확성: 0.6386999244142101
임계값: [0.09290083 1.18482914] 정확성: 0.6386999244142101
임계값: [0.34104825 1.2150897 ] 정확성: 0.6386999244142101
임계값: [0.13010319 1.89777107] 정확성: 0.23847316704459562
임계값: [1.6167947  1.93126407] 정확성: 0.13624338624338625
임계값: [0.19534423 0.60922754] 정확성: 0.63869

In [20]:
import statsmodels.api as sm

X_train_with_const = sm.add_constant(X_train)

model_sm = sm.OLS(y_train, X_train_with_const)
result = model_sm.fit()

print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                 credit   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     19.94
Date:                Wed, 08 May 2024   Prob (F-statistic):           1.71e-98
Time:                        14:23:09   Log-Likelihood:                -22263.
No. Observations:               21165   AIC:                         4.458e+04
Df Residuals:                   21136   BIC:                         4.482e+04
Df Model:                          28                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [21]:
# p-value가 0.05 미만인 변수들과 해당 변수들의 계수 출력
significant_variables = result.pvalues[result.pvalues < 0.05].index
significant_coefficients = result.params[significant_variables]

print("p-value가 0.05보다 작은 변수들과 해당 변수들의 계수:")
for variable, coefficient in zip(significant_variables, significant_coefficients):
    print(f"{variable}: {coefficient}")

p-value가 0.05보다 작은 변수들과 해당 변수들의 계수:
const: 0.6943686351844358
DAYS_BIRTH: 0.00221787266743043
email: 0.04143658023769883
begin_month: 0.006271332548178315
income_type_Commercial associate: 0.1446427692970847
income_type_State servant: 0.13700959838121715
income_type_Working: 0.12819141332787062
edu_type_Higher education: 0.08280635425803928
edu_type_Incomplete higher: 0.15358267665124736
edu_type_Lower secondary: 0.1530814082758964
edu_type_Secondary / secondary special: 0.12025453142451552
family_type_Civil marriage: 0.0979715127424778
family_type_Married: 0.15046280867935716
family_type_Separated: 0.1630821120415332
family_type_Single / not married: 0.16015347563229365
family_type_Widow: 0.12269872609114575
house_type_House / apartment: 0.14249861988386459
house_type_Municipal apartment: 0.173223311726662
house_type_Office apartment: 0.111152046283362
house_type_With parents: 0.1675919860852624
